In [177]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import asyncio
import httpx
from scipy.optimize import minimize

## Group Assignment
### Team Number: 10
### Team Member Names: David, Tanvi, Johan
### Team Strategy Chosen: Market Meet

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

-------------------------------------------------------------------------------------------------------------------------------------------
## Competition Goal: Market Meet

Goal: Try to have the 25-stock portfolio’s returns match the average of the TSX composite + S&P 500’s return (~0.370%) as accurately as possible, over the 5-day period (Nov. 24 to Nov. 28).

- **S&P/TSX Composite Index** (`^GSPTSE`)
- **S&P 500 Index** (`^GSPC`)

| Index | YTD Return | Daily Return (÷ 252)| 5-Day Estimated Return |
|:------|:-----------:|:--------------------:|:----------------------:|
| TSX Composite | 21.74% | 0.0863% | 0.43% |
| S&P 500 | 15.55% | 0.0617% | 0.31% |
| **Average (50/50)** | — | — | **≈ 0.370%** |

Use historical data (daily returns) from `yfinance` to estimate:
  - Mean returns
  - Volatility (standard deviation)
  - Sharpe ratio (risk-adjusted return)
  - Beta and alpha risk
  - Correlation with the benchmark
  - Idiosyncratic (residual) risk

Select 10–25 stocks (we aim for 25 if possible) that:
- Have **beta ≈ 1** and high correlation with the benchmark
- Are liquid (average daily volume ≥ 5,000 shares)
- Have sector diversification (no sector >40% of total value)
- Include at least one large-cap (> 10B CAD) and one small-cap (< $2B CAD)

Weights are between (100 / (2n))% and 15% (for 25 stocks, between 2% and 15%)
- Spend approximately **$1,000,000 CAD**, net of trading fees:
- Fees = min(2.15 USD, 0.001 USD * shares) per trade, applied to all purchases
-------------------------------------------------------------------------------------------------------------------------------------------

In [178]:
## === Temporary Test Ticker List (delete this when TA gives real Tickers.csv) ===

tickers_list = [
    "AAPL",      # US, tech
    "MSFT",      # US, tech
    "GOOGL",     # US, communications
    "AMZN",      # US, consumer cyclical
    "TSLA",      # Large-cap
    "NVDA",      # Large-cap, tech
    "JNJ",       # Large-cap, healthcare
    "WMT",       # Large-cap, consumer defensive
    "TD.TO",     # Large-cap, Canadian
    "SHOP.TO",   # Canadian tech
    "ENB.TO",    # Canadian energy
    "SU.TO",     # Canadian oil sands
    "L.TO",      # Consumer defensive Canada
    "TLRY",      # **Small-cap** Canadian (Tilray Brands)
    "AC.TO",     # Air Canada
    "BB.TO",     # BlackBerry
    "F",         # Ford
    "GM",        # General Motors
    "NKE",       # Nike
    "BABA",      # Alibaba
    "KSS" ,      # Kohl's
    "KO",        # Coca-Cola
]

In [179]:
# --- 1. Original Setup & Data Download (Synchronous) ---

sp500 = "^GSPC"
tsx   = "^GSPTSE"
start = "2024-10-01"
end   = "2025-9-30"

min_vol   = 5000
corr_min  = 0.20
small_cap = 2_000_000_000
large_cap = 10_000_000_000
earn_low  = datetime(2025, 11, 24).date()
earn_high = datetime(2025, 11, 28).date()



# Download price + volume data (Keep synchronous, as yf.download is efficient for bulk)
tickers_all = tickers_list + [sp500, tsx]
print("Starting synchronous download of price/volume data...")
data = yf.download(tickers_all, start=start, end=end, auto_adjust=False, progress=False)
close = data["Close"]
vol   = data["Volume"]
print("Price/Volume download complete.")

# Benchmark calculation (Unchanged)
bench = ((close[sp500] + close[tsx]) / 2).dropna()
bench_ret = bench.pct_change().dropna()
stocks = close[tickers_list].loc[bench_ret.index].dropna(how="all", axis=1)
rets = stocks.pct_change().dropna()

# --- 2. Asynchronous Metadata Fetching ---

# FX rate (Keep synchronous as it's a single, fast call)
fx_raw = yf.Ticker("CADUSD=X").history(period="1d")["Close"]
usd_to_cad = fx_raw.iloc[-1] if len(fx_raw) > 0 else 0.73  # fallback rate

async def async_get_meta(ticker):
    """Asynchronously fetches metadata for a single ticker."""
    t = ticker
    try:
        ticker_obj = yf.Ticker(t)
        
        # 1. Fetch info
        # Use a try/except for info specifically to avoid crashing the whole ticker
        try:
            info = await asyncio.to_thread(lambda: ticker_obj.info)
        except:
            info = {}
            
        sector = info.get("sector")
        industry = info.get("industry")
        mc_raw = info.get("marketCap", np.nan)
        
        # 2. Market Cap Conversion
        if isinstance(mc_raw, (int, float)) and not pd.isna(mc_raw):
            mc = mc_raw if t.endswith(".TO") else mc_raw / usd_to_cad 
        else:
            mc = np.nan

        # Define booleans safely
        small = mc < small_cap if not pd.isna(mc) else False
        large = mc > large_cap if not pd.isna(mc) else False

        # 3. Fetch Earnings Date (Robust 2-Step Check)
        earn = None
        
        # Attempt 1: Calendar (Fast)
        try:
            cal = await asyncio.to_thread(lambda: ticker_obj.calendar)
            if cal and "Earnings Date" in cal:
                dates = cal["Earnings Date"]
                if dates:
                    earn = dates[0]
        except Exception:
            pass 

        # Attempt 2: Full History (Backup)
        if earn is None:
            try:
                e_df = await asyncio.to_thread(lambda: ticker_obj.get_earnings_dates(limit=12))
                if e_df is not None and not e_df.empty:
                    future = [d.date() for d in e_df.index.to_pydatetime() if d.date() >= datetime.now().date()]
                    if future:
                        earn = min(future)
            except Exception:
                pass 

        return {
            "Ticker": t,
            "Sector": sector,
            "Industry": industry,
            "MarketCap": mc,
            "Small-cap": small,  
            "Large-cap": large,  
            "Earnings Date": earn
        }

    except Exception as e:
        # Fallback for total failure
        return {
            "Ticker": t,
            "Sector": np.nan, "Industry": np.nan, "MarketCap": np.nan,
            "Small-cap": False, 
            "Large-cap": False, 
            "Earnings Date": None
        }
    
async def fetch_all_metadata(tickers):
    print("Starting asynchronous metadata fetching with delay...")
    tasks = []
    for i, t in enumerate(tickers):
        # Create the metadata fetching task
        task = async_get_meta(t)
        tasks.append(task)

        # Pause execution for a short time every few requests
        if i > 0 and i % 5 == 0: # Example: pause every 5 tickers
            await asyncio.sleep(0.25) 

    # Now, gather all tasks that have been created
    results = await asyncio.gather(*tasks)
    print("Metadata fetching complete.")
    return results

# Run the asynchronous fetching operation
metadata_results = await fetch_all_metadata(stocks.columns)

# Convert results to DataFrame
meta = pd.DataFrame(metadata_results).set_index("Ticker")
meta = meta.loc[stocks.columns] # Re-align index order
meta["Small-cap"] = meta["Small-cap"].astype(bool)
meta["Large-cap"] = meta["Large-cap"].astype(bool)

# --- 3. Compute Metrics 

# --- Helper functions 
def avg_volume(series):
    """Average daily volume after removing months with <18 trading days."""
    s = series.dropna()
    if s.empty: return np.nan
    month = s.index.to_period("M")
    valid = month.value_counts()[lambda x: x >= 18].index
    return s[month.isin(valid)].mean()

def weekly_vol(r):
    """Weekly volatility computed from daily returns."""
    w = (1 + r).resample("W-FRI").prod() - 1
    w = w.dropna()
    return w.std() if not w.empty else np.nan

# --- Compute stock metrics 
metrics = pd.DataFrame(index=stocks.columns,
                       columns=["AvgVol", "StdDev (%)", "Covariance", "Beta", "Correlation", "WeeklyVol %","IdioVol %"],
                       dtype=float)
from datetime import date
for t in stocks.columns:
    r = rets[t].dropna()
    b = bench_ret.reindex(r.index).dropna()

    # Align
    idx = r.index.intersection(b.index)
    r = r.loc[idx]
    b = b.loc[idx]

    # Calculations
    av = avg_volume(vol[t].loc[start:end])
    sd = r.std() * 100
    cv = r.cov(b) * (100**2)
    beta = cv / (b.var() * (100**2)) if b.var() > 0 else np.nan
    corr = r.corr(b)
    wv = weekly_vol(r) * 100

    # --- NEW CALCULATION: Idiosyncratic Volatility ---
    # Formula: Std Dev of (Actual Return - Expected Return)
    # Expected Return = Alpha + (Beta * Benchmark Return)
    if not np.isnan(beta):
        # Calculate Alpha (Intercept)
        alpha = r.mean() - beta * b.mean()
        
        # Calculate Residuals (The noise not explained by the benchmark)
        residuals = r - (alpha + beta * b)
        
        # Calculate Idiosyncratic Volatility (Std Dev of residuals)
        iv = residuals.std() * 100
    else:
        iv = np.nan

    metrics.loc[t] = [av, sd, cv, beta, corr, wv, iv]

# --- Apply assignment filters & requirements ---

keep_vol = metrics["AvgVol"] >= min_vol
keep_corr = metrics["Correlation"] >= corr_min

keep_earn = pd.Series(True, index=metrics.index)
for t in metrics.index:
    e = meta.loc[t, "Earnings Date"]
    # Check if small-cap AND earnings date is within the forbidden window
    if meta.loc[t, "Small-cap"] and isinstance(e, date):
        if earn_low <= e <= earn_high:
            keep_earn[t] = False

mask = keep_vol & keep_corr & keep_earn

filtered = metrics[mask].join(meta, how="left")
filtered_tickers = list(filtered.index)

print("\n--- Results ---")
print("Original:", len(tickers_list))
print("After Filters:", len(filtered_tickers))
display(filtered.head(25))

# --- Benchmark reference data ---

bench_std = bench_ret.std() * 100
bench_week = weekly_vol(bench_ret) * 100

print("Benchmark Std Dev:", bench_std)
print("Benchmark Weekly Volatility:", bench_week)



Starting synchronous download of price/volume data...
Price/Volume download complete.
Starting asynchronous metadata fetching with delay...
Metadata fetching complete.

--- Results ---
Original: 22
After Filters: 19


,AvgVol,StdDev (%),Covariance,Beta,Correlation,WeeklyVol %,IdioVol %,Sector,Industry,MarketCap,Small-cap,Large-cap,Earnings Date
Ticker,,,,,,,,,,,,,
AAPL,5.384430e+07,2.085207,1.260429,1.542102,0.668600,4.464012,1.550607,Technology,Consumer Electronics,5.569451e+12,False,True,2026-01-29
MSFT,2.171526e+07,1.569207,0.849338,1.039143,0.598684,3.101843,1.256912,Technology,Software - Infrastructure,5.012677e+12,False,True,2026-01-28
GOOGL,3.418335e+07,2.058194,0.922382,1.128511,0.495703,4.239133,1.787525,Communication Services,Internet Content & Information,4.941642e+12,False,True,2025-10-29
AMZN,4.246964e+07,2.154998,1.262883,1.545104,0.648207,4.072332,1.640954,Consumer Cyclical,Internet Retail,3.271928e+12,False,True,2025-10-30
TSLA,1.002367e+08,4.517973,2.149786,2.630207,0.526319,8.780767,3.841574,Consumer Cyclical,Auto Manufacturers,1.852789e+12,False,True,2026-01-28
NVDA,2.302855e+08,3.139629,1.797425,2.199103,0.633242,6.419857,2.429928,Technology,Semiconductors,6.209543e+12,False,True,2025-11-19
WMT,1.795063e+07,1.527472,0.696616,0.852292,0.504449,3.434860,1.318883,Consumer Defensive,Discount Stores,1.204850e+12,False,True,2025-11-20
TD.TO,7.545756e+06,1.173627,0.438209,0.536137,0.412999,2.549827,1.068859,Financial Services,Banks - Diversified,1.974993e+11,False,True,2025-12-04
SHOP.TO,2.102246e+06,3.935635,2.372328,2.902481,0.666741,8.699801,2.933188,Technology,Software - Application,2.654718e+11,False,True,2025-11-04


Benchmark Std Dev: 0.902289793686018
Benchmark Weekly Volatility: 1.650726036520851


In [180]:
## === 4. Variable Stock Returns vs TSX/S&P 500 Benchmark ===

horizons = [5, 21, 63, 252]

# Prices aligned with benchmark dates
px = stocks[filtered_tickers].loc[stocks.index.intersection(bench.index)].dropna(how="all", axis=1)

def trailing_ret(series, d):
    s = series.dropna()
    if len(s) <= 1: return np.nan
    if len(s) <= d: return (s.iloc[-1] / s.iloc[0] - 1) * 100
    return (s.iloc[-1] / s.iloc[-(d+1)] - 1) * 100

def calc_avg_weekly(series):
    """Calculates the mean weekly return (%) over the available period."""
    # Resample to Weekly (Friday Close)
    w_prices = series.dropna().resample("W-FRI").last()
    # Calculate % Change
    w_rets = w_prices.pct_change().dropna()
    # Return Mean * 100
    return w_rets.mean() * 100

# Stock returns table
# UPDATED: Added "Avg Weekly (%)" to the column list
stock_returns = pd.DataFrame(index=px.columns,
                             columns=[f"{d}d" for d in horizons] + ["Avg Weekly (%)"])

for t in px.columns:
    for d in horizons:
        stock_returns.loc[t, f"{d}d"] = trailing_ret(px[t], d)
    
    # Calculate and store Avg Weekly Return for the stock
    stock_returns.loc[t, "Avg Weekly (%)"] = calc_avg_weekly(px[t])

# Benchmark returns table
bench_returns = pd.DataFrame(index=[f"{d}d" for d in horizons],
                             columns=["TSX/S&P Benchmark (%)"])

for d in horizons:
    bench_returns.loc[f"{d}d"] = trailing_ret(bench, d)

# UPDATED: Add the Benchmark's Avg Weekly Return as a new row
bench_returns.loc["Avg Weekly (%)"] = calc_avg_weekly(bench)

# Formatting
stock_returns = stock_returns.astype(float).round(6)
bench_returns = bench_returns.astype(float).round(6)

display(stock_returns)
display(bench_returns)

,5d,21d,63d,252d,Avg Weekly (%)
Ticker,,,,,
AAPL,-0.644328,9.404023,26.582086,12.192431,0.321954
MSFT,0.029150,0.973228,3.447575,23.366809,0.456850
GOOGL,-3.358015,15.313742,40.630409,47.142169,0.820783
AMZN,-2.398632,-4.071679,2.325904,20.247892,0.420005
TSLA,2.072730,28.102774,36.045796,77.981682,1.474072
NVDA,-0.958550,0.932457,17.307445,53.008000,0.929020
WMT,0.252871,7.252108,7.352641,28.161136,0.524903
TD.TO,1.414753,10.810815,11.713253,30.364891,0.515643
SHOP.TO,-4.515118,6.509913,34.530833,94.196390,1.556086


,TSX/S&P Benchmark (%)
5d,-0.053583
21d,4.855787
63d,11.370468
252d,23.166521
Avg Weekly (%),0.404268


In [181]:
#scoring
#beta -> x4
#market cap,returns,idiosyncratic volatility  -> x2 
# if earnings date within nov 24-28 AND small cap -> exclude


# === 5. Scoring & Normalization ===

# 1. Merge Dataframes
# We need 'Avg Weekly (%)' from stock_returns combined with Beta/Idio/Cap from filtered
df_score = filtered.join(stock_returns["Avg Weekly (%)"], how="inner")

# 2. Get Benchmark Target Return
target_return = bench_returns.loc["Avg Weekly (%)", "TSX/S&P Benchmark (%)"]
print(f"Target Benchmark Weekly Return: {target_return}%")

# --- SCORE CALCULATION ---

# A. Beta Score (40%)
# Logic: Minimize the distance (gap) between Stock Beta and 1.0
df_score["Beta_Gap"] = (df_score["Beta"] - 1.0).abs()
# Rank: Ascending=False means Smallest Gap gets Highest Score
df_score["Rank_Beta"] = df_score["Beta_Gap"].rank(ascending=False)

# B. Return Score (20%)
# Logic: Minimize the distance (gap) between Stock Return and Benchmark Return
df_score["Return_Gap"] = (df_score["Avg Weekly (%)"] - target_return).abs()
# Rank: Ascending=False means Smallest Gap gets Highest Score
df_score["Rank_Return"] = df_score["Return_Gap"].rank(ascending=False)

# C. Idiosyncratic Volatility Score (20%)
# Logic: Lower is Better (Minimize Noise)
# Rank: Ascending=False means Lowest Volatility gets Highest Score
df_score["Rank_Idio"] = df_score["IdioVol %"].rank(ascending=False)

# D. Market Cap Score (20%)
# Logic: Higher is Better (Liquidity & Index Drivers)
# Rank: Ascending=True means Highest Cap gets Highest Score
df_score["Rank_Cap"] = df_score["MarketCap"].rank(ascending=True)

# --- FINAL WEIGHTED SCORE ---
# Normalize ranks to a 0-100 scale relative to the number of stocks
n_stocks = len(df_score)

# Weights: Beta=4, Others=2 (Total = 10 parts)
# Formula: (4*Beta + 2*Cap + 2*Return + 2*Idio) / 10
df_score["Final_Score"] = (
    (0.40 * df_score["Rank_Beta"]) +
    (0.20 * df_score["Rank_Return"]) +
    (0.20 * df_score["Rank_Idio"]) +
    (0.20 * df_score["Rank_Cap"])
)

# Sort by Final Score
df_final = df_score.sort_values("Final_Score", ascending=False)

# --- SELECTION ---
# Select top 25
top_25 = df_final.head(25)

# Display Results
print("\n--- Top 25 Stock Selection ---")
cols_to_show = ["Beta", "Beta_Gap", "Avg Weekly (%)", "Return_Gap", "IdioVol %", "MarketCap", "Final_Score"]
display(top_25[cols_to_show])

# check if we have a large cap and small Cap
has_large = top_25["Large-cap"].any()
has_small = top_25["Small-cap"].any()

print(f"\nRequirement Check:")
print(f"Contains Large Cap (>10B): {has_large}")
print(f"Contains Small Cap (<2B):  {has_small}")

if not has_small:
    print("⚠️ WARNING: No Small Cap in Top 25. You must manually swap the 25th stock for the highest ranked Small Cap.")

Target Benchmark Weekly Return: 0.404268%

--- Top 25 Stock Selection ---


,Beta,Beta_Gap,Avg Weekly (%),Return_Gap,IdioVol %,MarketCap,Final_Score
Ticker,,,,,,,
MSFT,1.039143,0.039143,0.456850,0.052582,1.256912,5.012677e+12,17.4
WMT,0.852292,0.147708,0.524903,0.120635,1.318883,1.204850e+12,13.4
SU.TO,1.057056,0.057056,0.221105,0.183163,1.519296,7.642241e+10,13.2
ENB.TO,0.487625,0.512375,0.435988,0.031720,0.947196,1.486103e+11,12.8
GOOGL,1.128511,0.128511,0.820783,0.416515,1.787525,4.941642e+12,12.8
AAPL,1.542102,0.542102,0.321954,0.082314,1.550607,5.569451e+12,12.6
AMZN,1.545104,0.545104,0.420005,0.015737,1.640954,3.271928e+12,12.2
TD.TO,0.536137,0.463863,0.515643,0.111375,1.068859,1.974993e+11,12.2
BABA,1.035993,0.035993,1.077340,0.673072,2.965367,5.155048e+11,12.0



Requirement Check:
Contains Large Cap (>10B): True
Contains Small Cap (<2B):  True


In [182]:
def optimize_weights_for_top25(rets, stock_returns, bench_returns, top25, meta, df_score, lam=0.01):

    #get list of the selected tickers
    tickers = list(top25.index)

    #get final score values for these tickers
    score_vec = df_score.loc[tickers, "Final_Score"].astype(float)

    #normalize scores so they sum to one
    score_norm = score_vec / score_vec.sum()

    #get each stock's weekly return
    weekly_stock_ret = stock_returns.loc[tickers, "Avg Weekly (%)"].astype(float) / 100

    #get benchmark weekly return
    weekly_bench_ret = float(bench_returns.loc["Avg Weekly (%)", "TSX/S&P Benchmark (%)"]) / 100

    #objective combines strong return matching and score alignment
    def objective(weights):
        #compute portfolio weekly return
        port_ret = np.dot(weights, weekly_stock_ret)

        #we square the difference between portfolio return and benchmark return
        #then multiply by 1,000,000 so that even tiny differences become extremely important to the optimizer
        #which forecs the portfolio return to match the benchmark as close as it can
        track_err = 1_000_000 * (port_ret - weekly_bench_ret)**2

        #score alignment encourages high scoring stocks to have higher weights
        score_err = np.sum((weights - score_norm)**2)

        #final objective
        return track_err + lam * score_err

    #weights must sum to one
    def sum_to_one(weights):
        return np.sum(weights) - 1

    #sector must not exceed forty percent
    def sector_check(weights):
        sector_totals = {}
        for i, t in enumerate(tickers):
            sec = meta.loc[t, "Sector"]
            if sec not in sector_totals:
                sector_totals[sec] = 0
            sector_totals[sec] += weights[i]
        return min(0.40 - w for w in sector_totals.values())

    #ensure at least one large cap
    def large_cap_check(weights):
        total = 0
        for i, t in enumerate(tickers):
            if meta.loc[t, "MarketCap"] > 10_000_000_000:
                total += weights[i]
        return total - (1 / len(tickers))

    #ensure at least one small cap
    def small_cap_check(weights):
        total = 0
        for i, t in enumerate(tickers):
            if meta.loc[t, "MarketCap"] < 2_000_000_000:
                total += weights[i]
        return total - (1 / len(tickers))

    #constraint list
    constraints = [
        {'type': 'eq',  'fun': sum_to_one},
        {'type': 'ineq','fun': sector_check},
        {'type': 'ineq','fun': large_cap_check},
        {'type': 'ineq','fun': small_cap_check},
    ]

    #assignment weight bounds
    min_weight = 1 / (2 * len(tickers))
    bounds = [(min_weight, 0.15)] * len(tickers)

    #start near score distribution
    starting_guess = np.array(score_norm)

    #run optimizer
    result = minimize(
        fun=objective,
        x0=starting_guess,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 5000, "disp": True}
    )

    #normalize weights
    weights = result.x
    weights = weights / np.sum(weights)

    #compute final return and difference
    final_port_ret = np.dot(weights, weekly_stock_ret)
    difference = abs(final_port_ret - weekly_bench_ret)

    #print results
    print("\nOptimization:")
    print("Benchmark Weekly Return:", weekly_bench_ret)
    print("Portfolio Weekly Return:", final_port_ret)
    print("Absolute Difference:", difference)
    print("Sum of Weights:", np.sum(weights))

    return pd.Series(weights, index=tickers)

opt_weights = optimize_weights_for_top25(
    rets=rets,
    stock_returns=stock_returns,
    bench_returns=bench_returns,
    top25=top_25,
    meta=meta,
    df_score=df_score
)

display(opt_weights)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.06275320338248185
            Iterations: 2
            Function evaluations: 40
            Gradient evaluations: 2

Optimization:
Benchmark Weekly Return: 0.00404268
Portfolio Weekly Return: 0.004292154007618885
Absolute Difference: 0.0002494740076188851
Sum of Weights: 1.0


MSFT       0.026316
WMT        0.026316
SU.TO      0.150000
ENB.TO     0.026316
GOOGL      0.026316
AAPL       0.150000
AMZN       0.026316
TD.TO      0.026316
BABA       0.026316
F          0.026316
GM         0.026316
AC.TO      0.036842
NKE        0.150000
NVDA       0.026316
TSLA       0.026316
SHOP.TO    0.026316
BB.TO      0.026316
KSS        0.118421
TLRY       0.052632
dtype: float64